In [28]:
import json
import pickle
import re
import requests
import jieba
import nltk
import csv
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
def segmentSent(paragraph):
    sents = paragraph.split('\n')
    res = []
    for sent in sents:
        res.extend(re.findall(r"[^；。！？]+[；。！？]", sent))
    return [sent for sent in res if sent]
def loadWords(filename):
    words = set()
    with open(filename, 'r', encoding='utf-8') as inputFile:
        for line in inputFile:
            words.add(line.strip())
    return words
def getMention(tokenList, maxlen=5, minlen=2):
    mentionList = []
    length = len(tokenList)
    for begin in range(length):
        for end in range(begin + maxlen, begin + minlen - 1, -1):
            if end <= length:
                cand = ''.join(tokenList[begin:end])
                params = {'p': cand}
                try:
                    ret = requests.get('http://knowledgeworks.cn:30001/', params)
                    if ret.json() != ['Unknown Mention']:
                        print(cand)
                        mentionList.append(cand)
                        break
                except:
                    pass
    return mentionList

In [3]:
with open('data/cpc_reports.json', 'r', encoding='utf-8') as inputFile:
    jsonObj = json.load(inputFile)
with open('data/mentionLists.pkl', 'rb') as inputFile:
    mentionLists = pickle.load(inputFile)

In [5]:
tokenLists = {str(idx): [] for idx in range(13, 20)}
sentLists = {str(idx): [] for idx in range(13, 20)}

for idx, report in jsonObj.items():
    text = report['text']
    sents = segmentSent(text)
    sentLists[idx] = sents
    for sent in sents:
        tokenList = list(jieba.cut(sent))
        tokenLists[idx].extend(tokenList)
        # if idx == '13':
        #     mentionList = getMention(tokenList, maxlen=5, minlen=2)
        #     mentionLists[idx].extend(mentionList)

同志们


社会主义现代化建设


世界各国


这样的


东西方关系


和平共处五项原则


世界各国


世界和平


In [8]:
fds = {}
for idx, tokenList in tokenLists.items():
    fds[idx] = nltk.FreqDist(tuple(filter(lambda t: t.strip() != '', tokenList)))
stopwords = loadWords('data/chineseStopWords.txt')
for idx, fd in fds.items():
    for token in stopwords:
        if token in fd:
            fd.pop(token)
for idx, fd in fds.items():
    for token in fd:
        fd[token] /= len(tokenLists[idx])

        
def dumpFD(fd, filename):
    with open(filename, 'w', encoding='utf-8') as outputFile:
        for tf in fd.most_common(10000):
            outputFile.write('{}\t{}'.format(*tf) + '\n')
for idx, fd in fds.items():
    dumpFD(fd, idx + '.txt')

In [9]:
import math
def mergeFDs(fds):
    mergeFD = nltk.FreqDist()
    for fd in fds.values():
        for token, freq in fd.items():
            mergeFD[token] += freq
    return mergeFD
mergeFD = mergeFDs(fds)

idf = {}
for token in mergeFD:
    inDocCount = 0
    for fd in fds.values():
        if token in fd:
            inDocCount += 1
    idf[token] = math.log(len(fds) / float(inDocCount))

from collections import defaultdict
tfidf = defaultdict(nltk.FreqDist)
for idx, fd in fds.items():
    for token, freq in fd.items():
        tfidf[idx][token] = freq * idf[token]
for idx, values in tfidf.items():
    print(idx, ', '.join(list(map(lambda tf: tf[0], values.most_common(10)))))

13 达成, 协议, 真正, 缓和, 所作, 巨大, 制止, 早日, 地区冲突, 正是
14 历史, 主权, 人权, 方面, 独立, 建立, 我国, 友好关系, 包括, 得到
15 要, 国内, 出发, 大局, 制裁, 不会, 集团, 军事, 进行, 应
16 上应, 不应, 历史潮流, 应由, 政治经济, 光明, 福祉, 预见, 增加, 危害
17 紧密, 务实, 战略, 潮流, 区域合作, 恪守, 协作, 共赢, 帮助, 中国政府
18 全球性, 不要, 共, 赢, 发生, 持续, 平衡, 包容, 互鉴, 要
19 文明, 放弃, 气候变化, 能够, 构建, 理念, 治理, 超越, 人类, 共


In [10]:
mentionFDs = {}
for idx, mentionList in mentionLists.items():
    mentionFDs[idx] = nltk.FreqDist(filter(lambda m: m, map(lambda m: m.strip(), mentionList)))
for idx, fd in mentionFDs.items():
    total = sum(fd.values())
    for token in fd:
        fd[token] /= total
mergeMentionFD = mergeFDs(mentionFDs)

mentionIDF = {}
for token in mergeMentionFD:
    inDocCount = 0
    for fd in mentionFDs.values():
        if token in fd:
            inDocCount += 1
    mentionIDF[token] = math.log(len(fds) / float(inDocCount))

mentionTFIDF = defaultdict(nltk.FreqDist)
for idx, fd in mentionFDs.items():
    for token, freq in fd.items():
        mentionTFIDF[idx][token] = freq * mentionIDF[token]
for idx, values in mentionTFIDF.items():
    print(idx, ', '.join(list(map(lambda tf: tf[0], values.most_common(40)))))

14 国际新秩序, 新秩序, 他们的, 国际地位, 两极格局, 国际经济竞争, 经济环境, 对外关系, 并没有, 的尖端, 互不干涉内政, 国际经济新秩序, 国家垄断, 联合国成员国, 我们的, 新生活, 历史的经验, 得道多助，失道寡助, 世界各国, 国际社会, 中国外交政策, 军事集团, 第三世界国家, 国家发展, 不应, 正义的力量, 不可战胜, 新的, 联合国宪章, 和平与发展, 南北差距, 国际关系, 政治组织, 维护世界和平, 不称霸, 和平共处五项原则, 世界和平
15 军事集团, 威胁世界, 国际经济旧秩序, 还在, 我们都要, 是以, 发展战略, 生活方式, 国际上, 友好协商, 经济发展, 国际组织, 合作关系, 于人, 美好的, 新秩序, 世界各国, 新的, 冷战思维, 不公正, 不应, 他们的, 国际新秩序, 人民自己, 第三世界国家, 互利合作, 社会主义现代化建设, 和平国际, 中的, 和平与发展, 联合国宪章, 维护世界和平, 共同发展, 不称霸, 世界和平, 和平共处五项原则
16 仍是, 促进发展, 新安全观, 发展模式多样化, 不计较, 组织发展, 民间外交, 国际社会, 不可阻挡, 和平国际, 不公正, 不确定, 传统安全威胁, 中国外交政策, 人民自己, 国际合作, 正义的力量, 不可战胜, 世界多极化, 经济全球化, 新的, 共同发展, 也是, 非传统安全威胁, 繁荣的, 新秩序, 平等协商, 和平与发展, 南北差距, 国际关系民主化, 反对一切, 为善, 政治组织, 维护世界和平, 不称霸, 世界和平, 和平共处五项原则
17 全球经济失衡, 经济失衡, 战争手段, 当代中国, 中国的前途, 在一起, 战略抉择, 战略对话, 共赢, 大变革, 不可阻挡, 科技革命, 传统安全威胁, 人类和平, 和谐世界, 地球家园, 和平发展道路, 和平解决国际争端, 国防政策, 不对, 开放战略, 发展能力, 国家发展, 中国发展, 经济全球化, 南北差距, 国际关系, 时代主题, 非传统安全威胁, 也是, 繁荣的, 平等协商, 不分, 分大小, 自主选择, 国际社会, 投资自由化, 合作共赢, 国际秩序, 共同发展
18 中国, 就是要, 同时, 人类, 我们, 共赢, 世界经济, 正在发生, 新兴市场国家, 更多, 国际金融危机, 

In [18]:
api_key = 'i1V4s7C3R0ICSfgXaeiV0ZVnBbYXqamSnsOv6A3V'
api_url = 'http://api.ltp-cloud.com/analysis/'
sentParseLists = defaultdict(list)

data = {
    'api_key' : api_key,
    'pattern' : 'dp',
    'format' : 'json'
}

for idx, sents in sentLists.items():
    for sent in sents:
        data['text'] = sent
        result = requests.get(api_url, data)
        sentParseLists[idx].append(result.json())
    print('Finish {}'.format(idx))

Finish 13


Finish 14


Finish 15


Finish 16


Finish 17


Finish 18


Finish 19


In [ ]:
data_test = {
    'api_key' : api_key,
    'pattern' : 'ner',
    'text': '习近平在中国共产党第十九次全国代表大会上的报告',
    'format' : 'json'
}
requests.get(api_url, data_test).json()

NameError: name 'api_key' is not defined

In [14]:
import pickle
with open('data/parseLists.pkl', 'rb') as pklFile:
    sentParseLists = pickle.load(pklFile)
sentParseLists

defaultdict(list,
            {'14': [[[[{'cont': '在',
                  'id': 0,
                  'parent': 11,
                  'pos': 'p',
                  'relate': 'ADV'},
                 {'cont': '复杂',
                  'id': 1,
                  'parent': 2,
                  'pos': 'a',
                  'relate': 'ATT'},
                 {'cont': '多变',
                  'id': 2,
                  'parent': 5,
                  'pos': 'z',
                  'relate': 'ATT'},
                 {'cont': '的',
                  'id': 3,
                  'parent': 2,
                  'pos': 'u',
                  'relate': 'RAD'},
                 {'cont': '国际',
                  'id': 4,
                  'parent': 5,
                  'pos': 'n',
                  'relate': 'ATT'},
                 {'cont': '形势',
                  'id': 5,
                  'parent': 6,
                  'pos': 'n',
                  'relate': 'ATT'},
                 {'cont': '下',
          

In [83]:
sentParseLists['19'][10]

[[[{'cont': '要', 'id': 0, 'parent': 2, 'pos': 'v', 'relate': 'ADV'},
   {'cont': '相互', 'id': 1, 'parent': 2, 'pos': 'd', 'relate': 'ADV'},
   {'cont': '尊重', 'id': 2, 'parent': -1, 'pos': 'v', 'relate': 'HED'},
   {'cont': '、', 'id': 3, 'parent': 4, 'pos': 'wp', 'relate': 'WP'},
   {'cont': '平等', 'id': 4, 'parent': 5, 'pos': 'a', 'relate': 'ADV'},
   {'cont': '协商', 'id': 5, 'parent': 2, 'pos': 'v', 'relate': 'COO'},
   {'cont': '，', 'id': 6, 'parent': 2, 'pos': 'wp', 'relate': 'WP'},
   {'cont': '坚决', 'id': 7, 'parent': 8, 'pos': 'a', 'relate': 'ADV'},
   {'cont': '摒弃', 'id': 8, 'parent': 2, 'pos': 'v', 'relate': 'COO'},
   {'cont': '冷战', 'id': 9, 'parent': 10, 'pos': 'n', 'relate': 'ATT'},
   {'cont': '思维', 'id': 10, 'parent': 8, 'pos': 'n', 'relate': 'VOB'},
   {'cont': '和', 'id': 11, 'parent': 12, 'pos': 'c', 'relate': 'LAD'},
   {'cont': '强权政治', 'id': 12, 'parent': 10, 'pos': 'i', 'relate': 'COO'},
   {'cont': '，', 'id': 13, 'parent': 2, 'pos': 'wp', 'relate': 'WP'},
   {'cont': '走'

In [20]:
def collectRelation(sentParse):
    tree = defaultdict(list)
    parents = {}
    for tokenParse in sentParse[0][0]:
        parent = tokenParse['parent']
        son = tokenParse['id']
        tree[parent].append(son)
        parents[son] = parent
    return tree, parents
def collectTokenID(sentParse):
    idx2token = {-1: ''}
    for tokenParse in sentParse[0][0]:
        idx2token[int(tokenParse['id'])] = tokenParse['cont']
    return idx2token
def getSubTree(root, tree, maxdep=10):    
    def getSubTreeRecursively(node, tree, subTree, dep):
        if tree[node] and dep <= maxdep:
            for son in tree[node]:
                subTree.append(son)
                getSubTreeRecursively(son, tree, subTree, dep+1)
    dep = 0
    subTree = [root]
    getSubTreeRecursively(root, tree, subTree, dep+1)
    return subTree
def collectPunct(sentParse):
    punctIndices = []
    for tokenParse in sentParse[0][0]:
        if tokenParse['relate'] == 'WP' and tokenParse['cont'] not in ['“”（）：']:
            punctIndices.append(int(tokenParse['id']))
    return punctIndices
def cutWithPunct(tokenIdx, sent, punctIndices):
    beginIdx = 0
    for idx, tokenID in enumerate(sent):
        if tokenID in punctIndices:
            seg = sent[beginIdx:idx]
            beginIdx = idx + 1
            if tokenIdx in seg:
                return seg
    return sent

positiveVerbs = loadWords('data/positiveVerb.txt')
negativeVerbs = loadWords('data/negativeVerb.txt')

positiveRelation = defaultdict(list)
negativeRelation = defaultdict(list)
for idx, parseList in sentParseLists.items():
    for parse in parseList:
        tree, parents = collectRelation(parse)
        punctIndices = collectPunct(parse)
        # print(punctIndices)
        idx2token = collectTokenID(parse)
        for tokenIdx, token in sorted(idx2token.items()):
            if token == '不':
                subTreeLoose = getSubTree(parents[tokenIdx], tree, maxdep=100)
                mergeSubTree = sorted(set(subTreeLoose))
                subSent = cutWithPunct(tokenIdx, mergeSubTree, punctIndices)
                try:
                    phrase = ''.join(map(idx2token.get, subSent))
                    print(idx, phrase)
                except:
                    print(subSent)

14 不开
14 不参加任何军事集团
14 中国永远不称霸
14 永远不搞扩张
14 不应
15 不公正
15 、不合理
15 世界仍不安宁
15 不屈从于任何外来压力
15 不搞军事集团
15 不参加军备竞赛
15 不进行军事扩张
15 不应诉诸武力或以武力相威胁
15 不能
15 更不能
15 我们不把自己的社会制度和意识形态强加于人
15 不了
15 不搞对抗
15 不会
15 也永远不称霸
15 不会
16 不起来
16 不公正不合理的
16 不合理
16 不确定
16 世界还很不安宁
16 而不应把自己的意志强加于人
16 不应
16 不应
16 而不应诉诸武力或以武力相威胁
16 中国永远不称霸
16 永远不搞扩张
16 不计较社会制度和意识形态的差别
17 世界仍然很不安宁
17 坚持用和平方式而不是战争手段解决国际争端
17 中国将始终不渝走和平发展道路
17 国家不分大小
17 不干涉别国内部事务
17 不把自己的意志强加于人
17 不搞军备竞赛
17 不对任何国家构成军事威胁
17 永远不称霸
17 永远不搞扩张
17 中国将始终不渝奉行互利共赢的开放战略
17 不开
17 不开
18 世界仍然很不安宁
18 不稳定
18 不确定
18 不平衡
18 弱肉强食不是人共存之道
18 要和平不要战争
18 不要
18 国家不分大小
18 中国将始终不渝走和平发展道路
18 不干涉别国内政
18 永远不称霸
18 永远不搞扩张
18 中国将始终不渝奉行互利共赢的开放战略
19 世界面临的不稳定性不确定性
19 不能
19 不能
19 而不对抗
19 不结盟的
19 中国发展不对任何国家构成威胁
19 永远不称霸
19 永远不搞扩张
19 最不发达国家
19 国家不分大小


In [74]:
import jieba.posseg
wordcloud = WordCloud(font_path=r'C:\Windows\Fonts\simfang.ttf', width=1400, height=1400, margin=2,
                      max_words=50, prefer_horizontal=True)
text = open('data/国际形势.txt', 'r', encoding='utf-8').read()
pos_dict = {}

wcfd = nltk.FreqDist()
for w in jieba.posseg.cut(text):
    wcfd[w.word] += 1
    pos_dict[w.word] = w.flag
print(pos_dict['发展'])

for token in tuple(wcfd.keys()):
    if ('v' not in pos_dict[token] and 'n' not in pos_dict[token]) or token in stopwords or token in ['世界', '要', '不']:
        wcfd.pop(token)

wcfd.most_common(20)
wordcloud.generate_from_frequencies(wcfd)
wordcloud.to_file('wc.png')

vn


In [34]:
jieba.DICTIONARY

FreqDist({'一个': 0.00942631888960536,
          '一如既往': 0.0025795136850728876,
          '一带': 0.002530442326469848,
          '一律平等': 0.00319236142245591,
          '一新': 0.0017281617664789638,
          '一时': 0.00227858331271114,
          '一百多个': 0.0017281617664789638,
          '一系列': 0.0017281617664789638,
          '一致': 0.00227858331271114,
          '一贯主张': 0.00227858331271114,
          '一路': 0.002530442326469848,
          '一道': 0.0,
          '七十七国集团': 0.0017281617664789638,
          '上升': 0.0034193699562104028,
          '上同': 0.007902821463699517,
          '上应': 0.011429721874040018,
          '上海': 0.002453858952150458,
          '不': 0.008002090513301195,
          '不会': 0.00455716662542228,
          '不可': 0.004262950115155438,
          '不可逆转': 0.0031402542484727054,
          '不合理': 0.0033065287592643237,
          '不屈': 0.00227858331271114,
          '不应': 0.011429721874040018,
          '不得': 0.0017281617664789638,
          '不得人心': 0.0017281617664789638,
         